In [ ]:
!pip install nuscenes-devkit &> /dev/null  # Install nuScenes.
!pip install pyquaternion
! pip install pyntcloud
! pip install filterpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.3/346.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110458 sha256=d2879724cdfc50ca006242a88423d82ac074f728676a48972722b3ab206e9620
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy


In [ ]:
from IPython.display import display, Image
from IPython import display
import os
import shutil
from tqdm import tqdm
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import Box
from nuscenes.utils.geometry_utils import view_points
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
from pyquaternion import Quaternion
from nuscenes.utils.data_classes import LidarPointCloud, RadarPointCloud
from nuscenes.utils.geometry_utils import transform_matrix  # Importing the transform_matrix function
import json
import pyntcloud
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import RadarPointCloud
from nuscenes.utils.geometry_utils import view_points, transform_matrix
from filterpy.kalman import KalmanFilter
from filterpy.kalman import ExtendedKalmanFilter as EKF
from scipy.spatial.distance import mahalanobis

/usr/local/lib/python3.10/dist-packages/pyntcloud/utils/numba.py:5: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def groupby_count(xyz, indices, out):
/usr/local/lib/python3.10/dist-packages/pyntcloud/utils/numba.py:12: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def groupby_sum(xyz, indices, N, out):
/usr/local/lib/python3.10/dist-packages/pyntcloud/utils/nu

In [ ]:
%matplotlib inline
#from nuscenes.nuscenes import NuScenes

nusc = NuScenes(version='v1.0-mini', dataroot='/content/drive/MyDrive/centerfusion/CenterFusion/data/nuscenes', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 10.719 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


# **Actual Code**

In [ ]:
#function to project radar points onto image plane
def radar_to_camera_projection(nusc, radar_data_token, camera_data_token):
    # Load radar data
    radar_data = nusc.get('sample_data', radar_data_token)
    radar_pc = RadarPointCloud.from_file(nusc.get_sample_data_path(radar_data_token))
    radar_points = radar_pc.points[:3, :]  # Take only x, y, z coordinates

    # Load camera data
    camera_data = nusc.get('sample_data', camera_data_token)
    #print(camera_data)
    _,__,cam_intrinsic = nusc.get_sample_data(camera_data_token)

    #get required sensor data
    radar_cs_record = nusc.get('calibrated_sensor', radar_data['calibrated_sensor_token'])
    radar_ego_pose = nusc.get('ego_pose', radar_data['ego_pose_token'])
    cam_cs_record = nusc.get('calibrated_sensor', camera_data['calibrated_sensor_token'])

    #create tranform for radar to ego transform and tranform the objects from radar to ego frame
    radar_to_ego = transform_matrix(radar_cs_record['translation'], Quaternion(radar_cs_record['rotation']), inverse=False)
    radar_point_ego = np.dot(radar_to_ego, np.vstack((radar_points, np.ones(radar_points.shape[1]))))

    #create tranform for ego to camera transform and tranform the objects from ego to camera frame
    ego_to_cam = transform_matrix(cam_cs_record['translation'], Quaternion(cam_cs_record['rotation']), inverse=True)
    radar_points_camera = np.dot(ego_to_cam, radar_point_ego)

    # Project radar points onto the image plane
    radar_points_2d = view_points(radar_points_camera[:3, :], cam_intrinsic, normalize=True)

    return radar_points_2d

In [ ]:
#function to project 3d bboxes onto image
def project_boxes_to_2d(boxes, cam_intrinsic):
    projected_boxes = []
    for box in boxes:
        corners_3d = box.corners()
        corners_2d = view_points(corners_3d[:3, :], cam_intrinsic, normalize=True)[:2, :]
        min_x = min(corners_2d[0, :])
        min_y = min(corners_2d[1, :])
        max_x = max(corners_2d[0, :])
        max_y = max(corners_2d[1, :])
        center_x = (min_x + max_x) / 2
        center_y = (min_y + max_y) / 2
        width = max_x - min_x
        height = max_y - min_y
        projected_boxes.append([center_x, center_y,min_x,min_y,max_x,max_y, width, height])
    return projected_boxes

In [ ]:
#function for velocity association with bboxes
def associate_radar_points_to_boxes(radar_points, cam_detections):
    associated_velocities = []
    for detection in cam_detections:
        #print(detection)
        #print(radar_points)
        center_x,center_y,min_x,min_y,max_x,max_y, width, height = detection[0],detection[1],detection[2],detection[3],detection[4],detection[5],detection[6],detection[7]
        #print(center_x,center_y,min_x,min_y,max_x,max_y)
        box_center = np.array([center_x, center_y])

        object_points = []
        for x, y, vx, vy in radar_points:
            radar_point = np.array([x, y])
            if min_x <= x <= max_x and min_y <= y <= max_y:
              object_points.append([x,y,vx,vy])


        if len(object_points) > 0:
            object_points = np.array(object_points)
            velocities = object_points[:,2:4]  # Assuming vx, vy are in rows 3 and 4
            mean_velocity = np.mean(velocities, axis=0)
            #print(mean_velocity)
            associated_velocities.append([center_x, center_y,mean_velocity[0][0].tolist(),mean_velocity[1][0].tolist(), width, height])
            #print(mean_velocity)
        else:
            mean_velocity = np.array([0.0, 0.0])  # Default velocity for no points
            #print(mean_velocity)
            associated_velocities.append([center_x, center_y,mean_velocity[0].tolist(),mean_velocity[1].tolist(), width, height])
    return associated_velocities

In [ ]:
def initialize_ekf(x, y, vx, vy, w, h):
    dt = 0.5  # Time step
    ekf = EKF(dim_x=6, dim_z=4)  # 6 state variables, 4 measurements

    # State vector: [x, y, vx, vy, w, h]
    ekf.x = np.array([x, y, vx, vy, w, h])

    # State transition matrix
    ekf.F = np.array([[1, 0, dt, 0,  0, 0],
                      [0, 1, 0, dt,  0, 0],
                      [0, 0, 1,  0,  0, 0],
                      [0, 0, 0,  1,  0, 0],
                      [0, 0, 0,  0,  1, 0],
                      [0, 0, 0,  0,  0, 1]])

    # Measurement function matrix
    ekf.H = np.array([[1, 0, 0, 0, 0, 0],
                      [0, 1, 0, 0, 0, 0],
                      [0, 0, 1, 0, 0, 0],
                      [0, 0, 0, 1, 0, 0]])

    # Process noise covariance
    ekf.Q = np.eye(6) * 0.1  # Adjust for your process noise

    # Measurement noise covariance
    ekf.R = np.eye(4) * 1.0  # Adjust for your measurement noise

    # Initial state covariance
    ekf.P *= 1.0  # Adjust initial uncertainty

    return ekf

In [ ]:
from scipy.optimize import linear_sum_assignment

def euclidean_distance(detection, tracker):
    """
    Calculate the Euclidean distance between the detection and the tracker.
    Parameters:
    detection (np.ndarray): Detection coordinates [x, y]
    tracker (np.ndarray): Tracker coordinates [x, y]
    Returns:
    float: Euclidean distance
    """
    return np.sqrt((detection[0] - tracker[0]) ** 2 + (detection[1] - tracker[1]) ** 2)

def associate_detections_to_trackers_hungarian(detections, trackers, max_distance):
    """
    Assign detections to tracked objects using the Hungarian algorithm.
    Parameters:
    detections (np.ndarray): Array of detections in the format [[x, y], ...]
    trackers (np.ndarray): Array of trackers in the format [[x, y], ...]
    max_distance (float): Maximum distance to associate detection to tracker
    Returns:
    matches (np.ndarray): Array of matched indices [(detection_idx, tracker_idx), ...]
    unmatched_detections (np.ndarray): Array of unmatched detection indices
    unmatched_trackers (np.ndarray): Array of unmatched tracker indices
    """
    if len(trackers) == 0:
        return np.empty((0, 2), dtype=int), np.arange(len(detections)), np.empty((0, 2), dtype=int)

    distance_matrix = np.zeros((len(detections), len(trackers)), dtype=np.float32)

    for d, det in enumerate(detections):
        for t, trk in enumerate(trackers):
            distance_matrix[d, t] = euclidean_distance(det, trk)

    row_indices, col_indices = linear_sum_assignment(distance_matrix)

    matches = []
    unmatched_detections = []
    unmatched_trackers = []

    for d, det in enumerate(detections):
        if d not in row_indices:
            unmatched_detections.append(d)

    for t, trk in enumerate(trackers):
        if t not in col_indices:
            unmatched_trackers.append(t)

    for row, col in zip(row_indices, col_indices):
        if distance_matrix[row, col] < max_distance:
            matches.append((row, col))
        else:
            unmatched_detections.append(row)
            unmatched_trackers.append(col)

    matches = np.array(matches)
    unmatched_detections = np.array(unmatched_detections)
    unmatched_trackers = np.array(unmatched_trackers)

    return matches, unmatched_detections, unmatched_trackers


In [ ]:
object_id_counter = 0
object_trackers = {}

def initialize_trackers(detections):
    global object_id_counter
    global object_trackers # Declare object_trackers as global
    for detection in detections:
        x, y, vx, vy, w, h = detection
        ekf = initialize_ekf(x, y, vx, vy, w, h)
        object_trackers[object_id_counter] = {'ekf': ekf, 'bbox': [x, y], 'velocity': [vx, vy], "width" : w, "height": h}
        object_id_counter += 1

def measurement_function(state):
    return np.array([state[0], state[1]])  # Only x and y

def measurement_jacobian(state):
    return np.array([[1, 0, 0, 0, 0, 0],  # ∂x/∂x
                     [0, 1, 0, 0, 0, 0]])  # ∂y/∂y


def predict_and_associate(detections, max_distance=50):
    global object_id_counter
    global object_trackers # Declare object_trackers as global
    for obj_id, tracker in object_trackers.items():
        tracker['ekf'].predict()
        tracker['updated'] = False

    # Prepare detections and trackers for Hungarian association
    detections_positions = np.array([[x, y] for x, y, _, _ , ___, ______ in detections])
    trackers_positions = np.array([[tracker['bbox'][0], tracker['bbox'][1]] for tracker in object_trackers.values()])

    # Associate detections to trackers using Hungarian algorithm
    matches, unmatched_detections, unmatched_trackers = associate_detections_to_trackers_hungarian(detections_positions, trackers_positions, max_distance)
    print(matches)
    print(unmatched_detections)
    print(unmatched_trackers)


    # Update matched trackers
    for detection_idx, tracker_idx in matches:
        detection = detections[detection_idx]
        tracker_id = list(object_trackers.keys())[tracker_idx]
        x, y, vx, vy, w, h = detection
        # Calculate HJacobian and Hx using the current state of the tracker
        predicted_state = object_trackers[tracker_id]['ekf'].x_post  # Get the predicted state
        HJacobian = measurement_jacobian(predicted_state)
        Hx = measurement_function(predicted_state)
        R = np.eye(2) * 0.1  # Adjust the value 0.1 based on your sensor noise characteristics
        object_trackers[tracker_id]['ekf'].update(np.array([x, y]),measurement_jacobian,measurement_function, R = R)
        object_trackers[tracker_id]['bbox'] = [x, y]  # Update bounding box
        object_trackers[tracker_id]['velocity'] = [vx, vy]
        object_trackers[tracker_id]['width'] = w
        object_trackers[tracker_id]['height'] = h
        object_trackers[tracker_id]['updated'] = True

    # Initialize new trackers for unmatched detections
    for detection_idx in unmatched_detections:
        x, y, vx, vy, w, h = detections[detection_idx]
        new_ekf = initialize_ekf(x, y, vx, vy, w, h)
        object_trackers[object_id_counter] = {'ekf': new_ekf, 'bbox': [x, y], 'velocity': [vx, vy], "width": w, "height": h, 'updated': True}
        object_id_counter += 1

    # Remove trackers that were not updated
    object_trackers = {obj_id: tracker for obj_id, tracker in object_trackers.items() if tracker['updated']}

In [ ]:
def process_scenes(scenes, image_output_dir, radar_output_dir, label_output_dir):
    processed_images = set()
    processed_radar_PC = set()

    for scene in tqdm(scenes):
        first_sample_token = scene['first_sample_token']
        sample = nusc.get('sample', first_sample_token)

        ground_truth = []
        while sample:
            frame_data = {
                "frame": sample['token'],
                "Image_path": [],
                "Radar_path": [],
                "camera_translation": [],
                "camera_rotation": [],
                "camera_intrinsic": [],
                "radar_translation": [],
                "radar_rotation": [],
                "objects": []
            }

            cam_data_token = sample['data']['CAM_FRONT']
            cam_data = nusc.get('sample_data', cam_data_token)
            cam_data_path, cam_boxes, cam_intrinsics = nusc.get_sample_data(cam_data_token)
            image_basename = os.path.basename(cam_data_path)
            cam_cs_record = nusc.get('calibrated_sensor', cam_data['calibrated_sensor_token'])
            cam_translation = cam_cs_record['translation']
            cam_rotation = cam_cs_record['rotation']

            if image_basename not in processed_images:
                image_output_path = os.path.join(image_output_dir, image_basename)
                shutil.copy(cam_data_path, image_output_path)
                frame_data["Image_path"] = image_output_path
                frame_data["camera_translation"] = cam_translation
                frame_data["camera_rotation"] = cam_rotation
                frame_data["camera_intrinsic"] = cam_intrinsics.tolist()
                processed_images.add(image_basename)
            else:
                frame_data["Image_path"] = os.path.join(image_output_dir, image_basename)
                frame_data["camera_translation"] = cam_translation
                frame_data["camera_rotation"] = cam_rotation
                frame_data["camera_intrinsic"] = cam_intrinsics.tolist()

            # Read radar data
            radar_data_token = sample['data']['RADAR_FRONT']
            radar_data = nusc.get('sample_data', radar_data_token)
            radar_data_path, boxes, radar_intrinsics = nusc.get_sample_data(radar_data_token)
            radar_pc = RadarPointCloud.from_file(radar_data_path)
            radar_cs_record = nusc.get('calibrated_sensor', radar_data['calibrated_sensor_token'])
            radar_translation = radar_cs_record['translation']
            radar_rotation = radar_cs_record['rotation']

            # Save radar point cloud to dataset
            radar_basename = os.path.basename(radar_data_path)
            if radar_basename not in processed_radar_PC:
                radar_output_path = os.path.join(radar_output_dir, radar_basename)
                shutil.copy(radar_data_path, radar_output_path)
                frame_data["Radar_path"] = radar_output_path
                frame_data["radar_translation"] = radar_translation
                frame_data["radar_rotation"] = radar_rotation
                processed_radar_PC.add(radar_basename)
            else:
                frame_data["Radar_path"] = os.path.join(radar_output_dir, radar_basename)
                frame_data["radar_translation"] = radar_translation
                frame_data["radar_rotation"] = radar_rotation

            radar_points_2d = radar_to_camera_projection(nusc, radar_data_token=radar_data_token, camera_data_token=cam_data_token)
            radar_detections = radar_points_2d[:2, :].T
            radar_points = []
            radar_points.append(np.vstack((radar_points_2d[:2, :], radar_pc.points[6:8, :])))
            radar_points_array = np.array(radar_points)
            radar_points_array = radar_points_array.T

            # Process camera boxes to get 2D coordinates
            cam_detections = project_boxes_to_2d(boxes=cam_boxes, cam_intrinsic=cam_intrinsics)
            print(cam_detections)

            # Get the associated velocity for the boxes
            detections_with_velocity = associate_radar_points_to_boxes(radar_points=radar_points_array, cam_detections=cam_detections)
            print(detections_with_velocity)

            if len(ground_truth) == 0:
                print("Initializing trackers")
                initialize_trackers(detections=detections_with_velocity)
            else:
                print("Predicting and associating")
                predict_and_associate(detections=detections_with_velocity)

            for obj_id, tracker in object_trackers.items():
                bbox = tracker['bbox']
                velocity = tracker['velocity']
                width = tracker['width']
                height = tracker['height']
                frame_data["objects"].append({"id": obj_id, "bbox": bbox, "velocity": velocity, "width": width, "height": height})

            ground_truth.append(frame_data)
            print(f"Processed frame: {sample['token']}, total ground truth frames: {len(ground_truth)}")

            next_sample_token = sample['next']
            if next_sample_token:
                sample = nusc.get('sample', next_sample_token)
            else:
                break

        with open('ground_truth_with_velocity.json', 'w') as f:
            json.dump(ground_truth, f)


In [ ]:
def process_scenes___(scenes, image_output_dir, radar_output_dir, label_output_dir):

    for scene in tqdm(scenes):
        first_sample_token = scene['first_sample_token']
        sample = nusc.get('sample', first_sample_token)

        ground_truth = []
        while sample:
            frame_data = {"frame": sample['token'], "Image_path":[], "Radar_path":[], "camera_traslation": [], "camera_rotation" : [], "camera_intrinsic": [],"radar_translation" : [], "radar_rotation": [], "objects": []}
            processed_images = set()
            processed_radar_PC = set()
            cam_data_token = sample['data']['CAM_FRONT']
            cam_data = nusc.get('sample_data',cam_data_token)
            cam_data_path,cam_boxes,cam_intrinsics = nusc.get_sample_data(cam_data_token)
            image_basename = os.path.basename(cam_data_path)
            cam_cs_record = nusc.get('calibrated_sensor', cam_data['calibrated_sensor_token'])
            cam_translation = cam_cs_record['translation']
            cam_rotation = cam_cs_record['rotation']

            if image_basename not in processed_images:
               image_ouput_path = os.path.join(image_output_dir,image_basename)
               shutil.copy(cam_data_path,image_ouput_path)
               frame_data["Image_path"] = image_ouput_path
               frame_data["camera_traslation"] = cam_translation
               frame_data["camera_rotation"] = cam_rotation
               frame_data["camera_intrinsic"] = cam_intrinsics.tolist()

            #read radar data
            radar_data_token = sample['data']['RADAR_FRONT']
            radar_data = nusc.get('sample_data',radar_data_token)
            radar_data_path,boxes,radar_intrinsics = nusc.get_sample_data(radar_data_token)
            radar_pc = RadarPointCloud.from_file(radar_data_path)
            radar_cs_record = nusc.get('calibrated_sensor', radar_data['calibrated_sensor_token'])
            radar_translation = radar_cs_record['translation']
            radar_rotation = radar_cs_record['rotation']

            #save radar pointcloud to dataset
            radar_basename = os.path.basename(radar_data_path)
            if radar_basename not in processed_radar_PC:
               radar_output_path = os.path.join(radar_output_dir,radar_basename)
               shutil.copy(radar_data_path,radar_output_path)
               frame_data["Radar_path"] = radar_output_path
               frame_data["radar_translation"] = radar_translation
               frame_data["radar_rotation"] = radar_rotation

            radar_points_2d = radar_to_camera_projection(nusc, radar_data_token=radar_data_token, camera_data_token=cam_data_token)
            radardetections = radar_points_2d[:2,:].T
            radarpoints = []
            radarpoints.append(np.vstack((radar_points_2d[:2, :], radar_pc.points[6:8, :])))
            radarpointsarray = np.array(radarpoints)
            radarpointsarray = radarpointsarray.T

            #process camboxes to get 2d coordinates
            cam_detections = project_boxes_to_2d(boxes= cam_boxes, cam_intrinsic= cam_intrinsics)
            print(cam_detections)

            #get the associated velocity for the boxes
            detections_with_velocity = associate_radar_points_to_boxes(radar_points=radarpointsarray,cam_detections= cam_detections)
            print(detections_with_velocity)

            #print(len(ground_truth))
            #print(ground_truth)
            if len(ground_truth) == 0:
                print("Initializing trackers")
                initialize_trackers(detections=detections_with_velocity)
            else:
                print("Predicting and associating")
                predict_and_associate(detections=detections_with_velocity)

            for obj_id, tracker in object_trackers.items():
                bbox = tracker['bbox']
                velocity = tracker['velocity']
                frame_data["objects"].append({"id": obj_id, "bbox": bbox, "velocity": velocity,})


            ground_truth.append(frame_data)
            #print(frame_data)
            #print(f"Processed frame: {sample['token']}, total ground truth frames: {len(ground_truth)}")

            next_sample_token = sample['next']
            if next_sample_token:
                sample = nusc.get('sample', next_sample_token)
            else:
                break

        with open('ground_truth_with_velocity.json', 'w') as f:
                 json.dump(ground_truth, f)


In [ ]:
#Split scenes into train and val
import random

# Define the ratio of validation scenes
validation_ratio = 0.1  # 20% of scenes for validation, 80% for training

# Get the list of all scenes
all_scenes = nusc.scene

# Shuffle the scenes randomly
#random.shuffle(all_scenes)

# Calculate the number of scenes for validation
num_validation_scenes = int(validation_ratio * len(all_scenes))

# Split scenes into validation and training sets
val_scenes = all_scenes[7:8]
#val_scenes = all_scenes[:num_validation_scenes]
train_scenes = all_scenes[num_validation_scenes:]

#val_scenes
#train_scenes

In [ ]:
all_scenes

[{'token': 'cc8c0bf57f984915a77078b10eb33198',
  'log_token': '7e25a2c8ea1f41c5b0da1e69ecfa71a2',
  'nbr_samples': 39,
  'first_sample_token': 'ca9a282c9e77460f8360f564131a8af5',
  'last_sample_token': 'ed5fc18c31904f96a8f0dbb99ff069c0',
  'name': 'scene-0061',
  'description': 'Parked truck, construction, intersection, turn left, following a van'},
 {'token': 'fcbccedd61424f1b85dcbf8f897f9754',
  'log_token': '53cf9c55dd8644bea67b9f009fc1ee38',
  'nbr_samples': 40,
  'first_sample_token': '3e8750f331d7499e9b5123e9eb70f2e2',
  'last_sample_token': '281b92269fd648d4b52d06ac06ca6d65',
  'name': 'scene-0103',
  'description': 'Many peds right, wait for turning car, long bike rack left, cyclist'},
 {'token': '6f83169d067343658251f72e1dd17dbc',
  'log_token': '881dd2f8aaec49b681eb54be5bf3b3e2',
  'nbr_samples': 41,
  'first_sample_token': '8687ba92abd3406aa797115b874ebeba',
  'last_sample_token': 'dcbe451d383e450786aaad04ab9d3790',
  'name': 'scene-0553',
  'description': 'Wait at intersect

In [ ]:
val_scenes

[{'token': 'd25718445d89453381c659b9c8734939',
  'log_token': '8fefc430cbfa4c2191978c0df302eb98',
  'nbr_samples': 41,
  'first_sample_token': '5998b71b64c146769bde1d5430741381',
  'last_sample_token': 'ee9b3f38dadd40588f43ba590c4751b9',
  'name': 'scene-1077',
  'description': 'Night, big street, bus stop, high speed, construction vehicle'}]

In [ ]:
output_dir = '/content/drive/MyDrive/testdata3'
image_output_dir = os.path.join(output_dir, 'images')
label_output_dir = os.path.join(output_dir, 'labels')
radar_output_dir = os.path.join(output_dir, 'radar')

In [ ]:
os.makedirs(image_output_dir, exist_ok=True)
os.makedirs(label_output_dir, exist_ok=True)
os.makedirs(radar_output_dir, exist_ok=True)

In [ ]:
# Process val scenes
process_scenes(val_scenes, image_output_dir, radar_output_dir, label_output_dir)

  0%|          | 0/1 [00:00<?, ?it/s]

[[369.9824960307616, 515.0423682731204, 312.1164832899228, 491.1234005085079, 427.8485087716004, 538.9613360377327, 115.73202548167757, 47.83793552922481], [521.9078231686217, 537.683545518091, 414.54229721106435, 494.64103772910806, 629.2733491261789, 580.726053307074, 214.73105191511456, 86.08501557796598], [82.63892699330661, 525.0394132452054, -6.221003981128647, 490.6106317455533, 171.49885796774188, 559.4681947448574, 177.71986194887054, 68.85756299930415], [108.73079830953951, 588.912842497997, -82.49432988272216, 491.1427364579293, 299.9559265018012, 686.6829485380646, 382.4502563845233, 195.5402120801353], [204.92671022967616, 516.4898344982619, 142.0181018730098, 489.3231155031003, 267.8353185863425, 543.6565534934235, 125.8172167133327, 54.333437990323205]]
[[369.9824960307616, 515.0423682731204, 6.416666666666667, -35.416666666666664, 115.73202548167757, 47.83793552922481], [521.9078231686217, 537.683545518091, 1.65, -28.35, 214.73105191511456, 86.08501557796598], [82.63892

100%|██████████| 1/1 [01:00<00:00, 60.57s/it]

[[1264.7733730215703, 512.6724231303199, 1225.7204848147117, 491.4890571519149, 1303.8262612284288, 533.8557891087249, 78.1057764137172, 42.36673195681004], [946.5517075263485, 510.1216188911569, 935.2715789152408, 501.15581651388396, 957.8318361374563, 519.0874212684298, 22.56025722221557, 17.931604754545845], [849.2403056004525, 537.4522517498715, 782.3073231580978, 484.1117336686236, 916.1732880428071, 590.7927698311195, 133.86596488470934, 106.6810361624959], [1155.5189450873486, 507.4437205816973, 1127.4154937795668, 490.1616129541129, 1183.6223963951302, 524.7258282092816, 56.20690261556342, 34.56421525516873]]
[[1264.7733730215703, 512.6724231303199, -18.375, 0.6875, 78.1057764137172, 42.36673195681004], [946.5517075263485, 510.1216188911569, 0.0, 0.0, 22.56025722221557, 17.931604754545845], [849.2403056004525, 537.4522517498715, -9.916666666666666, 1.1666666666666667, 133.86596488470934, 106.6810361624959], [1155.5189450873486, 507.4437205816973, -19.5, 0.8333333333333334, 56.2

In [ ]:
#formatting Json file to make it readable
def format_json_file(input_file_path, output_file_path=None):
    # Read the JSON data from the input file
    with open(input_file_path, 'r') as input_file:
        data = json.load(input_file)

    # Format the JSON data
    formatted_json = json.dumps(data, indent=4)

    # Print formatted JSON data
    print(formatted_json)

    # Write the formatted JSON data to the output file (if provided)
    if output_file_path:
        with open(output_file_path, 'w') as output_file:
            output_file.write(formatted_json)

In [ ]:
input_file_path = '/content/ground_truth_with_velocity.json'  # Path to your input JSON file
output_file_path = '//content/drive/MyDrive/testdata3/ground_truth.json'  # Path to your output JSON file (optional)
format_json_file(input_file_path, output_file_path)

[
    {
        "frame": "5998b71b64c146769bde1d5430741381",
        "Image_path": "/content/drive/MyDrive/testdata3/images/n015-2018-11-21-19-38-26+0800__CAM_FRONT__1542800367912460.jpg",
        "Radar_path": "/content/drive/MyDrive/testdata3/radar/n015-2018-11-21-19-38-26+0800__RADAR_FRONT__1542800367978326.pcd",
        "camera_translation": [
            1.70079118954,
            0.0159456324149,
            1.51095763913
        ],
        "camera_rotation": [
            0.4998015430569128,
            -0.5030316162024876,
            0.4997798114386805,
            -0.49737083824542755
        ],
        "camera_intrinsic": [
            [
                1266.417203046554,
                0.0,
                816.2670197447984
            ],
            [
                0.0,
                1266.417203046554,
                491.50706579294757
            ],
            [
                0.0,
                0.0,
                1.0
            ]
        ],
        "radar_tr

In [ ]:
def reassign_object_ids(input_file_path, output_file_path):
    # Read the JSON data from the input file
    with open(input_file_path, 'r') as input_file:
        data = json.load(input_file)

    # Reassign object IDs to start from 0
    next_id = 0
    id_mapping = {}

    for frame_data in data:
        for obj in frame_data['objects']:
            original_id = obj['id']
            if original_id not in id_mapping:
                id_mapping[original_id] = next_id
                next_id += 1
            obj['id'] = id_mapping[original_id]

    # Write the updated JSON data to the output file
    with open(output_file_path, 'w') as output_file:
        json.dump(data, output_file, indent=4)

In [ ]:
input_file_path = '/content/drive/MyDrive/testdata2/ground_truth.json'  # Path to your input JSON file
output_file_path = '/content/drive/MyDrive/testdata2/ground_truth_reindexed.json'  # Path to your output JSON file
reassign_object_ids(input_file_path, output_file_path)